# Case 1 - Discover the Pattern of Success

## A. Import Library & Data

In [25]:
from openpyxl import load_workbook
import duckdb
import pandas as pd
import sys
import os

DB_FILE = 'employee_data.duckdb'
EXCEL_FILE = 'Study Case DA.xlsx'

wb = load_workbook(EXCEL_FILE, read_only=True)
sheet_names = wb.sheetnames
print(f"Sheets in '{EXCEL_FILE}': {sheet_names}")

Sheets in 'Study Case DA.xlsx': ['Talent Variable (TV) & Talent G', 'dim_companies', 'dim_areas', 'dim_positions', 'dim_departments', 'dim_divisions', 'dim_directorates', 'dim_grades', 'dim_education', 'dim_majors', 'dim_competency_pillars', 'employees', 'profiles_psych', 'papi_scores', 'strengths', 'performance_yearly', 'competencies_yearly']


## B. Data Preparation

### 1. Use SQL script and prepare some variables

In [15]:
SQL_SCHEMA_SCRIPT = '''
-- Dimension Tables
CREATE TABLE dim_companies (
  company_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_areas (
  area_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_positions (
  position_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_departments (
  department_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_divisions (
  division_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_directorates (
  directorate_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_grades (
  grade_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_education (
  education_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_majors (
  major_id BIGINT PRIMARY KEY,
  name TEXT UNIQUE NOT NULL
);

CREATE TABLE dim_competency_pillars (
  pillar_code VARCHAR(3) PRIMARY KEY,
  pillar_label TEXT NOT NULL
);

-- Fact & Profile Tables
CREATE TABLE employees (
  employee_id TEXT PRIMARY KEY,
  fullname TEXT,
  nip TEXT,
  company_id BIGINT REFERENCES dim_companies(company_id),
  area_id BIGINT REFERENCES dim_areas(area_id),
  position_id BIGINT REFERENCES dim_positions(position_id),
  department_id BIGINT REFERENCES dim_departments(department_id),
  division_id BIGINT REFERENCES dim_divisions(division_id),
  directorate_id BIGINT REFERENCES dim_directorates(directorate_id),
  grade_id BIGINT REFERENCES dim_grades(grade_id),
  education_id BIGINT REFERENCES dim_education(education_id),
  major_id BIGINT REFERENCES dim_majors(major_id),
  years_of_service_months BIGINT
);

CREATE TABLE profiles_psych (
  employee_id TEXT PRIMARY KEY REFERENCES employees(employee_id),
  pauli NUMERIC,
  faxtor NUMERIC,
  disc TEXT,
  disc_word TEXT,
  mbti TEXT,
  iq NUMERIC,
  gtq INT,
  tiki INT
);

CREATE TABLE papi_scores (
  employee_id TEXT REFERENCES employees(employee_id),
  scale_code TEXT,
  score INT,
  PRIMARY KEY (employee_id, scale_code)
);

CREATE TABLE strengths (
  employee_id TEXT REFERENCES employees(employee_id),
  rank INT,
  theme TEXT,
  PRIMARY KEY (employee_id, rank)
);

CREATE TABLE performance_yearly (
  employee_id TEXT REFERENCES employees(employee_id),
  year INT,
  rating INT,
  PRIMARY KEY (employee_id, year)
);

CREATE TABLE competencies_yearly (
  employee_id TEXT REFERENCES employees(employee_id),
  pillar_code VARCHAR(3) REFERENCES dim_competency_pillars(pillar_code),
  year INT,
  score INT,
  PRIMARY KEY (employee_id, pillar_code, year)
);

-- Indexes (Non-Primary Key)
CREATE INDEX performance_yearly_index_3 ON performance_yearly (year);
CREATE INDEX competencies_yearly_index_5 ON competencies_yearly (pillar_code, year);

-- Comments
COMMENT ON TABLE dim_competency_pillars IS 'Codes: GDR, CEX, IDS, QDD, STO, SEA, VCU, LIE, FTC, CSI';
COMMENT ON TABLE strengths IS 'CliftonStrengths rank 1..14';
'''

In [26]:
# Variable for data processing (table names, primary key variable, and string variable)
TABLE_NAMES = [
    'dim_companies',
    'dim_areas',
    'dim_positions',
    'dim_departments',
    'dim_divisions',
    'dim_directorates',
    'dim_grades',
    'dim_education',
    'dim_majors',
    'dim_competency_pillars',
    'employees',
    'profiles_psych',
    'papi_scores',
    'strengths',
    'performance_yearly',
    'competencies_yearly'
]

PK_COLUMNS = {
    'employees': ['employee_id'],
    'profiles_psych': ['employee_id'],
    'dim_competency_pillars': ['pillar_code'],
    'papi_scores': ['employee_id', 'scale_code'],
    'strengths': ['employee_id', 'rank'],
    'performance_yearly': ['employee_id', 'year'],
    'competencies_yearly': ['employee_id', 'pillar_code', 'year']
}

DTYPE_OVERRIDES = {
    'employees': {'employee_id': str, 'nip': str},
    'profiles_psych': {'employee_id': str},
    'papi_scores': {'employee_id': str, 'scale_code': str},
    'strengths': {'employee_id': str},
    'performance_yearly': {'employee_id': str},
    'competencies_yearly': {'employee_id': str, 'pillar_code': str},
    'dim_competency_pillars': {'pillar_code': str}
}

### 2. Define function for data extract

In [ ]:
def create_schema(conn):
    """Executes the main SQL script to create all tables."""
    try:
        print("Connecting to database and creating schema...")
        conn.execute(SQL_SCHEMA_SCRIPT)
        print("Schema created successfully.")
    except duckdb.Error as e:
        print(f"Error creating schema: {e}")
        print("The database might already exist. If so, delete the file '{DB_FILE}' and try again.")
        sys.exit(1)

def load_data_from_excel(conn):
    """Loops through table names, reading from Excel sheets and inserting into DuckDB."""
    print(f"\nStarting data load from '{EXCEL_FILE}'...")
    
    for table_name in TABLE_NAMES:
        print(f"  - Loading data for table: '{table_name}'")
        try:
            # Get specific dtypes for this table
            dtypes = DTYPE_OVERRIDES.get(table_name, None)
            
            # Read the corresponding sheet from the Excel file
            df = pd.read_excel(EXCEL_FILE, sheet_name=table_name, engine='openpyxl', dtype=dtypes)
            
            if df.empty:
                print(f"    ...Sheet '{table_name}' is empty. Skipping.")
                continue

            # Handle NULLs in Primary Key columns
            if table_name in PK_COLUMNS:
                pk_cols = PK_COLUMNS[table_name]
                initial_rows = len(df)
                # Drop rows where any of the PK columns are null
                df.dropna(subset=pk_cols, inplace=True)
                dropped_rows = initial_rows - len(df)
                if dropped_rows > 0:
                    print(f"    ...Dropped {dropped_rows} rows with NULL values in primary key columns: {pk_cols}")

            if df.empty:
                print(f"    ...No valid data left after cleaning. Skipping.")
                continue

            # Register the DataFrame as a temporary view in DuckDB
            conn.register('temp_table', df)
            
            # Use INSERT BY NAME
            # This maps columns by name instead of by position.
            conn.execute(f"INSERT INTO {table_name} BY NAME SELECT * FROM temp_table")
            
            # Clean up the temporary view
            conn.unregister('temp_table')
            
            print(f"    ...Success: Loaded {len(df)} rows into '{table_name}'.")
            
        except FileNotFoundError:
            print(f"Error: The file '{EXCEL_FILE}' was not found.")
            sys.exit(1)
        except Exception as e:
            # This often happens if the sheet doesn't exist
            print(f"    ...Error loading sheet '{table_name}': {e}")
            print("    ...Skipping this table. Please check your Excel file.")

def verify_data(conn):
    """Runs a few simple queries to confirm data was loaded."""
    print("\nVerifying data load...")
    try:
        # Check employee count
        employee_count = conn.execute("SELECT COUNT(*) FROM employees").fetchone()[0]
        print(f"Total employees loaded: {employee_count}")
        
        # Check company count
        company_count = conn.execute("SELECT COUNT(*) FROM dim_companies").fetchone()[0]
        print(f"Total companies loaded: {company_count}")
        
        # Sample data from employees
        print("\nSample 5 employees:")
        print(conn.execute("SELECT employee_id, fullname, nip FROM employees LIMIT 5").df())
        
        # Sample data from performance
        print("\nSample 5 performance records:")
        print(conn.execute("SELECT * FROM performance_yearly LIMIT 5").df())
        
    except duckdb.Error as e:
        print(f"Error during verification: {e}")
        print("Verification failed. Data may be incomplete.")

### 3. Use function to extracting data from sources

In [24]:
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"{DB_FILE} deleted successfully.")
else:
    print(f"{DB_FILE} does not exist.")

with duckdb.connect(database=DB_FILE, read_only=False) as conn:
    create_schema(conn)
    load_data_from_excel(conn)
    verify_data(conn)

print(f"\nAll done! Your database is saved as '{DB_FILE}'.")

employee_data.duckdb deleted successfully.
Connecting to database and creating schema...
Schema created successfully.

Starting data load from 'Study Case DA.xlsx'...
  - Loading data for table: 'dim_companies'
    ...Success: Loaded 4 rows into 'dim_companies'.
  - Loading data for table: 'dim_areas'
    ...Success: Loaded 4 rows into 'dim_areas'.
  - Loading data for table: 'dim_positions'
    ...Success: Loaded 6 rows into 'dim_positions'.
  - Loading data for table: 'dim_departments'
    ...Success: Loaded 6 rows into 'dim_departments'.
  - Loading data for table: 'dim_divisions'
    ...Success: Loaded 5 rows into 'dim_divisions'.
  - Loading data for table: 'dim_directorates'
    ...Success: Loaded 3 rows into 'dim_directorates'.
  - Loading data for table: 'dim_grades'
    ...Success: Loaded 3 rows into 'dim_grades'.
  - Loading data for table: 'dim_education'
    ...Success: Loaded 4 rows into 'dim_education'.
  - Loading data for table: 'dim_majors'
    ...Success: Loaded 6 row

## C. Data Exploration